**Zhilin Guo (zg2358) & Fangpu He (fh2398)  
4701 AI Kaggle Competition**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
dir_path = '/kaggle/input/f2019-aihw7/'
for dirname, _, filenames in os.walk(dir_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# These paths are unique to Kaggle, obviously. Use your local path or colab path, depending on which you're using.
train_x = np.load(dir_path+'mnist-train-images.npy')
train_y = np.load(dir_path+'mnist-train-labels.npy')
val_x = np.load(dir_path+'mnist-val-images.npy')
val_y = np.load(dir_path+'mnist-val-labels.npy')

scan_train_x = np.load(dir_path+'scan-train-images.npy')
scan_train_y = np.load(dir_path+'scan-train-labels.npy')
scan_test_x = np.load(dir_path+'scan-test-images.npy')

# Verify that their shapes are what we expect
print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("val_x shape:", val_x.shape)
print("val_y shape:", val_y.shape)
print()
print("scan_train_x shape:", scan_train_x.shape)
print("scan_train_y shape:", scan_train_y.shape)
print("scan_test_x shape:", scan_test_x.shape)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)

# show each image, and make each title the label
# these are grayscale images so use appropriate heatmap
ax1.imshow(train_x[4701], cmap=plt.get_cmap('gray'))
ax1.set_title(str(train_y[4701]))
ax2.imshow(train_x[4702], cmap=plt.get_cmap('gray'))
ax2.set_title(str(train_y[4702]))
ax3.imshow(train_x[4703], cmap=plt.get_cmap('gray'))
ax3.set_title(str(train_y[4703]))

fig.show()

In [ ]:
# print data type
print("Data type:", train_x.dtype)
# just to make sure, print the min/max too
print("Data min:", np.amin(train_x[4701]))
print("Data max:", np.amax(train_x[4701]))

In [ ]:
print("Data type:", train_y.dtype)

In [ ]:
fig, ax = plt.subplots()
ax.hist(train_y, bins=range(11))
ax.set_xticks(range(10))
ax.set_title("MNIST Training Set Class Distribution")

fig.show()

In [ ]:
import keras
from keras.datasets import mnist

#(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.concatenate((train_x, scan_train_x))
y_train = np.concatenate((train_y, scan_train_y))

x_test = val_x
y_test = val_y

# This is a quirk of keras; since the images are grayscale,
# we need to add an axis so the shape is (60000, 28, 28, 1)
# instead of (60000, 28, 28)

x_train = x_train[:,:,:,np.newaxis]
x_test = x_test[:,:,:,np.newaxis]

# We're also going to convert 0~255 to 0~1 float.
x_train = x_train.astype(np.float)
x_test = x_test.astype(np.float)

x_train /= 255
x_test /= 255

# Finally, the classes need to be one-hot encoded.
# That is:
# 0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0]
# 1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0]
# etc.
# This is to match what the network will output - 
# there are 10 nodes at the end, each with its own
# confidence of its class. The ground truth should be
# 100% confidence of the true label.

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
#                        Remember these?

# By the way, we really like powers of 2 for the number
# of nodes at each layer.

# model = Sequential([
#     # input layer, 16 conv (spatial) perceptrons of size (3,3)
#     # image shape is (28, 28, 1). If it was color it'd be (28, 28, 3)
#     Conv2D(30, (5,5), activation='relu', input_shape=(28, 28, 1)),
#     # Now for the max pooling to make the size smaller
#     MaxPooling2D(pool_size=(2,2)),
#     # Flatten before sending to Dense (2D to 1D)
#     Flatten(),
#     # Output layer with 10 nodes for 10 classes, with softmax
#     Dense(10, activation='softmax')
# ])

#our own model with sequential layers
model = Sequential([
    # input layer, 16 conv (spatial) perceptrons of size (3,3)
    # image shape is (28, 28, 1). If it was color it'd be (28, 28, 3)
    Conv2D(64, kernel_size=(6,6), activation='relu', input_shape=(28, 28, 1)),
    # Now for the max pooling to make the size smaller
    MaxPooling2D(),
    
    # another Conv2D layer
    Conv2D(32, (5,5), activation='relu'),
    MaxPooling2D(),
    
    
    # prevent overfitting
    Dropout(0.1),
    
    # Flatten before sending to Dense
    Flatten(),
    
    # multiple dense layers
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    
    # Output layer with 10 nodes for 10 classes, with softmax
    Dense(10, activation='softmax')
])


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.SGD(),
             metrics=['accuracy'])

In [ ]:
import time
start = time.time()
model.fit(x_train,        # training data
          y_train,        # training labels
          batch_size=300,  # how many training examples you want to give at once
          verbose=1,      # print progress in console
          validation_data=(x_test, y_test),  # validation data to check generalization
          epochs= 1000)       # how many times to go through the entire training set
end = time.time()
print("Training took", end-start, "seconds.")

In [ ]:
import csv

scan_test_x = np.load(dir_path+'scan-test-images.npy')
scan_test_x = scan_test_x[:,:,:,np.newaxis]
scan_test_x = scan_test_x.astype(np.float)
scan_test_x /= 255

with open('output.csv', mode='w') as csv_file:
    res = model.predict(scan_test_x)
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Id", "Category"])
    id = 0
    for pred_res in res:
        pred_num = np.argmax(pred_res)
        csv_writer.writerow([id, pred_num])
        id += 1

In [ ]:
scan_test_x = np.load(dir_path+'scan-test-images.npy')

fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)

# show each image, and make each title the label
# these are grayscale images so use appropriate heatmap
ax1.imshow(scan_test_x[0], cmap=plt.get_cmap('gray'))
ax2.imshow(scan_test_x[1], cmap=plt.get_cmap('gray'))
ax3.imshow(scan_test_x[2], cmap=plt.get_cmap('gray'))

fig.show()